In [ ]:
import nltk
from nltk.corpus import treebank
from nltk.tag.hmm import HiddenMarkovModelTagger
from nltk.tokenize import word_tokenize
from nltk.tokenize import treebank
import yaml
import pandas as pd
import re
from yaml import safe_load
from sqlalchemy.dialects import postgresql
from sqlalchemy import Integer, Numeric, String, DateTime, create_engine
import psycopg2
from nltk.tokenize import TreebankWordTokenizer
twt = TreebankWordTokenizer()
import random
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
import os
from dotenv import load_dotenv
load_dotenv()

In [505]:
pd.set_option('max_colwidth', 1000)

In [749]:
input_sentence = 'Turkish official says death toll from earthquake is now at 284, bringing overall death toll in Turkey and Syria to 568'

In [798]:
def is_inside(token_span, regex_spans):
    span_membership = []
    for regex_span in regex_spans:
        if (token_span[0] >= regex_span[0]) & (token_span[1] <= regex_span[1]):
            span_membership.append(True) 
        else:
            span_membership.append(False)
    return span_membership

def make_unicode(inp):
    if type(inp) != unicode:
        inp =  inp.decode('utf-8')
    return inp

def kw_query(query_word):
    """
    Grab a random bit of text that references your keyword query

    This is problematic because some returns only have the country at the beginning of the text; we want more flavor than that.
    """
    
    connection = psycopg2.connect(user=user,
                              password=password,
                              host=host,
                              port=port,
                              database=database)
    cursor = connection.cursor()
    
    sql = f'''SELECT title FROM articles WHERE title ~ '(?!^)\y{query_word.replace("'","")}\y' limit 1'''

    cursor.execute(sql)
    # print("Selecting rows from mobile table using cursor.fetchall")
    result = cursor.fetchone()
    connection.close()
    # print(type(result))
    # print(result)
    # print(len(result))
    if result != None:
        return result[0]
    else:
        return None

def flatten_extend(matrix):
     flat_list = []
     for row in matrix:
         flat_list.extend(row)
     return flat_list

def annotate(input_sentence):
    """
    1. REGEX search the input text for country names, getting the appropriate span-match
    2. Word tokenize, and span tokenize
    3. Compare the span-tokenized spans to the regex spans
    4. Poop out annotated docs
    """
    ## Regex search for country names
    regex_hits = {}
    for match in regex_query.finditer(input_sentence):
        regex_hits[(match.start(), match.end())] = match.group()

    ## Tokenization
    tokens = [x for x in twt.tokenize(input_sentence)]
    token_spans = [x for x in twt.span_tokenize(input_sentence)]

    ## Stop early if there's no country mention
    if len(regex_hits)<1:
        return [(token, 'o') for token in tokens]
    
    ## Compare the REGEX and Tokenized spans    
    xyz = []
    for token_span in token_spans:
        if any(is_inside(token_span, list(regex_hits.keys()))):
            xyz.append('Country')
        else:
            xyz.append("o")
            
    return list(zip(tokens, xyz))

def get_random_titles(limit=25):
    connection = psycopg2.connect(user=user,
                              password=password,
                              host=host,
                              port=port,
                              database=database)
    cursor = connection.cursor()
    
    sql = f'''SELECT title FROM articles ORDER BY random() LIMIT {limit};'''
    
    cursor.execute(sql)
    # print("Selecting rows from mobile table using cursor.fetchall")
    result = cursor.fetchall()
    connection.close()

    return result

from nltk.tag.hmm import HiddenMarkovModelTagger

def train_hmm(train, dev):
    """
    Train an HMM tagger on the training set provided
    """
    
    hmm_tagger = HiddenMarkovModelTagger.train(train, dev)
    
    return hmm_tagger

## Connect to DB

In [5]:
database = os.getenv('DATABASE')
host = os.getenv('DBHOST')
port = os.getenv('DBPORT')
user = os.getenv('DBUSER')
password = os.getenv('DBPASSWORD')

In [8]:
conn_string = fr'postgresql+psycopg2://{user}:{password}@{host}/{database}'
db = create_engine(conn_string)
conn = db.connect()

## Load and munge data

In [800]:
df['example']

,aliases,alpha2,alpha3,fifa,ioc,iso_name,numeric,official,short,emoji,shortcode,simple,example,annotated
AD,[Principality of the Valleys of Andorra],AD,AND,AND,AND,Andorra,20,Principality of Andorra,Andorra,🇦🇩,:flag-ad:,NaN,"Pique to move, rename FC Andorra after stadium row with govt.","[(Pique, o), (to, o), (move, o), (,, o), (rename, o), (FC, o), (Andorra, Country), (after, o), (stadium, o), (row, o), (with, o), (govt, o), (., o)]"
AE,"[Emirates, UAE]",AE,ARE,UAE,UAE,United Arab Emirates,784,United Arab Emirates,United Arab Emirates,🇦🇪,:flag-ae:,NaN,Ashes hero Andrew Flintoff to continue coaching stint after horror Top Gear smash by mentoring England Lions on their training camp in United Arab Emirates,"[(Ashes, o), (hero, o), (Andrew, o), (Flintoff, o), (to, o), (continue, o), (coaching, o), (stint, o), (after, o), (horror, o), (Top, o), (Gear, o), (smash, o), (by, o), (mentoring, o), (England, o), (Lions, o), (on, o), (their, o), (training, o), (camp, o), (in, o), (United, Country), (Arab, Country), (Emirates, Country)]"
AF,NaN,AF,AFG,AFG,AFG,Afghanistan,4,Islamic Republic of Afghanistan,Afghanistan,🇦🇫,:flag-af:,NaN,"Afghanistan: Latest news, breaking stories and comment","[(Afghanistan, Country), (:, o), (Latest, o), (news, o), (,, o), (breaking, o), (stories, o), (and, o), (comment, o)]"
AG,NaN,AG,ATG,ATG,ANT,Antigua and Barbuda,28,Antigua and Barbuda,Antigua and Barbuda,🇦🇬,:flag-ag:,NaN,"Antigua and Barbuda is preparing to auction off an 'abandoned' $81 million superyacht linked to a Russian oligarch, reports say","[(Antigua, Country), (and, Country), (Barbuda, Country), (is, o), (preparing, o), (to, o), (auction, o), (off, o), (an, o), ('abandoned, o), (', o), ($, o), (81, o), (million, o), (superyacht, o), (linked, o), (to, o), (a, o), (Russian, Country), (oligarch, o), (,, o), (reports, o), (say, o)]"
AI,NaN,AI,AIA,None,None,Anguilla,660,Anguilla,Anguilla,🇦🇮,:flag-ai:,NaN,Real Housewives Of New York City: Jenna Lyons and Brynn Whitfield outrageously flirt with each other during group trip to Anguilla,"[(Real, o), (Housewives, o), (Of, o), (New, o), (York, o), (City, o), (:, o), (Jenna, o), (Lyons, o), (and, o), (Brynn, o), (Whitfield, o), (outrageously, o), (flirt, o), (with, o), (each, o), (other, o), (during, o), (group, o), (trip, o), (to, o), (Anguilla, Country)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YE,NaN,YE,YEM,YEM,YEM,Yemen,887,Republic of Yemen,Yemen,🇾🇪,:flag-ye:,NaN,Saudi Arabia to grant Yemen $1.2 billion in economic aid,"[(Saudi, Country), (Arabia, o), (to, o), (grant, o), (Yemen, Country), ($, o), (1.2, o), (billion, o), (in, o), (economic, o), (aid, o)]"
YT,NaN,YT,MYT,None,None,Mayotte,175,Department of Mayotte,Mayotte,🇾🇹,:flag-yt:,NaN,"Boat capsizes off Madagascar on way to Mayotte, 22 killed","[(Boat, o), (capsizes, o), (off, o), (Madagascar, Country), (on, o), (way, o), (to, o), (Mayotte, Country), (,, o), (22, o), (killed, o)]"
ZA,NaN,ZA,ZAF,RSA,RSA,South Africa,710,Republic of South Africa,South Africa,🇿🇦,:flag-za:,NaN,Drakensberg Park: The First Shamans of South Africa | Arkeo Ep27 I Documentary,"[(Drakensberg, o), (Park, o), (:, o), (The, o), (First, o), (Shamans, o), (of, o), (South, Country), (Africa, Country), (|, o), (Arkeo, o), (Ep27, o), (I, o), (Documentary, o)]"
ZM,NaN,ZM,ZMB,ZAM,ZAM,Zambia,894,Republic of Zambia,Zambia,🇿🇲,:flag-zm:,NaN,"Exclusive: Zambia sent bondholders debt restructuring proposal, finance ministry says","[(Exclusive, o), (:, o), (Zambia, Country), (sent, o), (bondholders, o), (debt, o), (restructuring, o), (proposal, o), (,, o), (finance, o), (ministry, o), (says, o)]"


In [11]:
with open("en.yml", "r") as f:
    try:
        countries = (yaml.safe_load(f))
    except yaml.YAMLError as exc:
        print(exc)

In [32]:
leader_df = pd.read_csv(r"world-leaders-social-media.csv")

In [22]:
no_aliases = []
for country in countries:
    if 'aliases' in country:
        pass
    else:
        no_aliases.append(country)

In [24]:
for x in no_aliases:
    countries[x]['aliases'] = []

In [34]:
df = df.T

In [801]:
df['example'] = df['iso_name'].progress_apply(kw_query)

100%|████████████████████████████████████████████████████████████████████████████████| 205/205 [02:04<00:00,  1.65it/s]
C:\Users\Dan\AppData\Local\Temp\ipykernel_22572\2633083810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['example'] = df['iso_name'].progress_apply(kw_query)


In [146]:
df = df[df['example'].str.len()>0]

In [ ]:
demonym_df = pd.read_excel("demonyms.xlsx")

adj = flatten_extend([i.split(', ') for i in demonym_df['Adjectivals'].tolist()])
dem = flatten_extend([i.split(', ') for i in demonym_df['Demonyms'].tolist()])

In [802]:
country_lookup = df['iso_name'].to_list()
country_lookup.extend(df['short'].to_list())

country_lookup.extend(flatten_extend(df[df['aliases'].notna()]['aliases'].to_list()))
country_lookup.extend(adj)
country_lookup.extend(dem)

bonus_context = ['US', 'U.S.', 'USA', 'U.S.A.' 'England', 'Herzegovina', 'British', 'Kurd', 'Kurdish', 'Washington', 'Beijing', 'Moscow', 'London',
                'Gaza', 'Palestine', 'West Bank', 'Levant', 'NKorea', 'SKorea', "Uyghurs","Uighurs", "Uygurs", "Uigurs","Uyghur","Uighur", "Uygur", 
                 "Uigur", "Native Americans", "Native American", 'Hezbollah', 'ISIS', 'ISIL', 'Daesh', 'Obama', 'Biden', 'Mike Pence', 'Donald Trump',
                'Putin', 'Xi Jinping', 'Syria', 'Syrian', 'Zelensky', 'Berlin', 'Madrid', 'Paris', 'Korea']
country_lookup.extend(bonus_context)

country_lookup = [x.strip() for x in country_lookup]

print("Before dedup:",len(country_lookup))
country_lookup = list(set(country_lookup))
print("After dedup:",len(country_lookup))

Before dedup: 1155
After dedup: 881


In [803]:
## Todo: escape periods. For some reason this is really hard
country_lookup = [x.replace('.', r'.') for x in country_lookup]
country_lookup.sort()
country_query = [fr"\b{x}\b" for x in country_lookup]
regex_query = re.compile("|".join(country_query))

In [804]:
'Putin' in country_lookup

True

## Apply annotations/Make training data

In [808]:
df['example'] = df['example'].astype(str)

C:\Users\Dan\AppData\Local\Temp\ipykernel_22572\2011254448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['example'] = df['example'].astype(str)


In [809]:
df['annotated'] = df['example'].progress_apply(annotate)
annotated = df['annotated'].to_list()
train_data = get_random_titles(50000)
df3 = pd.DataFrame(train_data, columns=['text'])
df3 = df3.drop_duplicates('text')
print(len(df3))
df3['annotated'] = df3['text'].progress_apply(annotate)
df3 = df3[df3['annotated'].astype(str).str.contains('Country')]
annotated2 = df3['annotated'].to_list()
annotated.extend(annotated2)

100%|██████████████████████████████████████████████████████████████████████████████| 205/205 [00:00<00:00, 4181.41it/s]
C:\Users\Dan\AppData\Local\Temp\ipykernel_22572\1966908333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['annotated'] = df['example'].progress_apply(annotate)


47580


100%|██████████████████████████████████████████████████████████████████████████| 47580/47580 [00:11<00:00, 4304.70it/s]


In [810]:
len(annotated)

12075

In [811]:
train, dev = train_test_split(annotated, test_size=0.2, random_state=42)

dataset = {"train": train, "dev":dev}

## Train Hidden Markov Model

In [812]:
hmm_tagger = train_hmm(dataset["train"], dataset["dev"])

accuracy over 35818 tokens: 98.84


## Validate

In [813]:
testoo = get_random_titles(500)
df4 = pd.DataFrame(testoo, columns=['text'])

In [814]:
df4 = df4.drop_duplicates('text')
len(df4)

488

In [815]:
df4['annotate'] = df4['text'].progress_apply(annotate)
df4['hmm'] = df4['text'].progress_apply(lambda x: hmm_tagger.tag(twt.tokenize(x)))

100%|███████████████████████████████████████████████████████████████████████████████| 488/488 [00:01<00:00, 382.82it/s]


In [816]:
len(df4[df4['annotate']!=df4['hmm']])

115

In [817]:
df4[df4['annotate']!=df4['hmm']].sample(5)

,text,annotate,hmm
115,Katharine McPhee shows off her long legs in playful social media post calling out AMC Theatres: 'Where's my concert movie deal?',"[(Katharine, o), (McPhee, o), (shows, o), (off, o), (her, o), (long, o), (legs, o), (in, o), (playful, o), (social, o), (media, o), (post, o), (calling, o), (out, o), (AMC, o), (Theatres, o), (:, o), ('Where, o), ('s, o), (my, o), (concert, o), (movie, o), (deal, o), (?, o), (', o)]","[(Katharine, Country), (McPhee, o), (shows, o), (off, o), (her, o), (long, o), (legs, o), (in, o), (playful, o), (social, o), (media, o), (post, o), (calling, o), (out, o), (AMC, o), (Theatres, o), (:, o), ('Where, o), ('s, o), (my, o), (concert, o), (movie, o), (deal, o), (?, o), (', o)]"
282,Christina Ricci seemingly calls out Ashton Kutcher and his wife Mila Kunis amid Danny Masterson rape case controversy: 'Believe victims',"[(Christina, o), (Ricci, o), (seemingly, o), (calls, o), (out, o), (Ashton, o), (Kutcher, o), (and, o), (his, o), (wife, o), (Mila, o), (Kunis, o), (amid, o), (Danny, o), (Masterson, o), (rape, o), (case, o), (controversy, o), (:, o), ('Believe, o), (victims, o), (', o)]","[(Christina, Country), (Ricci, o), (seemingly, o), (calls, o), (out, o), (Ashton, o), (Kutcher, o), (and, o), (his, o), (wife, o), (Mila, o), (Kunis, o), (amid, o), (Danny, o), (Masterson, o), (rape, o), (case, o), (controversy, o), (:, o), ('Believe, o), (victims, o), (', o)]"
356,Nitrous oxide: MPs approve bill to ban laughing gas possession,"[(Nitrous, o), (oxide, o), (:, o), (MPs, o), (approve, o), (bill, o), (to, o), (ban, o), (laughing, o), (gas, o), (possession, o)]","[(Nitrous, Country), (oxide, o), (:, o), (MPs, o), (approve, o), (bill, o), (to, o), (ban, o), (laughing, o), (gas, o), (possession, o)]"
288,Trump Says House Destroyed Jan. 6 Documents,"[(Trump, o), (Says, o), (House, o), (Destroyed, o), (Jan., o), (6, o), (Documents, o)]","[(Trump, Country), (Says, o), (House, o), (Destroyed, o), (Jan., o), (6, o), (Documents, o)]"
301,Eighth Covid wave strikes Australia: Chief medical officer Paul Kelly reassures strain is less 'severe' than predecessors,"[(Eighth, o), (Covid, o), (wave, o), (strikes, o), (Australia, Country), (:, o), (Chief, o), (medical, o), (officer, o), (Paul, o), (Kelly, o), (reassures, o), (strain, o), (is, o), (less, o), ('severe, o), (', o), (than, o), (predecessors, o)]","[(Eighth, Country), (Covid, o), (wave, o), (strikes, o), (Australia, Country), (:, o), (Chief, o), (medical, o), (officer, o), (Paul, o), (Kelly, o), (reassures, o), (strain, o), (is, o), (less, o), ('severe, o), (', o), (than, o), (predecessors, o)]"


In [818]:
df4.sample(5)

,text,annotate,hmm
252,First pill for fecal transplants wins FDA approval,"[(First, o), (pill, o), (for, o), (fecal, o), (transplants, o), (wins, o), (FDA, o), (approval, o)]","[(First, o), (pill, o), (for, o), (fecal, o), (transplants, o), (wins, o), (FDA, o), (approval, o)]"
182,Deutsche Bank makes the highest S&P 500 forecast on Wall Street — and says that may be too conservative,"[(Deutsche, o), (Bank, o), (makes, o), (the, o), (highest, o), (S, o), (&, o), (P, o), (500, o), (forecast, o), (on, o), (Wall, o), (Street, o), (—, o), (and, o), (says, o), (that, o), (may, o), (be, o), (too, o), (conservative, o)]","[(Deutsche, Country), (Bank, o), (makes, o), (the, o), (highest, o), (S, o), (&, o), (P, o), (500, o), (forecast, o), (on, o), (Wall, o), (Street, o), (—, o), (and, o), (says, o), (that, o), (may, o), (be, o), (too, o), (conservative, o)]"
361,Brit in Japan describes house shaking in quake,"[(Brit, o), (in, o), (Japan, Country), (describes, o), (house, o), (shaking, o), (in, o), (quake, o)]","[(Brit, o), (in, o), (Japan, Country), (describes, o), (house, o), (shaking, o), (in, o), (quake, o)]"
24,Inside the Commando Raids Unnerving Russia in Crimea,"[(Inside, o), (the, o), (Commando, o), (Raids, o), (Unnerving, o), (Russia, Country), (in, o), (Crimea, o)]","[(Inside, o), (the, o), (Commando, o), (Raids, o), (Unnerving, o), (Russia, Country), (in, o), (Crimea, o)]"
107,"Highlights from the third GOP debate, in 3 minutes","[(Highlights, o), (from, o), (the, o), (third, o), (GOP, o), (debate, o), (,, o), (in, o), (3, o), (minutes, o)]","[(Highlights, o), (from, o), (the, o), (third, o), (GOP, o), (debate, o), (,, o), (in, o), (3, o), (minutes, o)]"


## Explore how it broke
- Periods in acronyms are a bear to deal with in regex

In [819]:
hmm_tagger.tag(twt.tokenize("U.S. officials found eating cheezits under the bleachers"))

[('U.S.', 'o'),
 ('officials', 'o'),
 ('found', 'o'),
 ('eating', 'o'),
 ('cheezits', 'o'),
 ('under', 'o'),
 ('the', 'o'),
 ('bleachers', 'o')]

In [820]:
annotate("U.S. officials found eating cheezits under the bleachers")

[('U.S.', 'o'),
 ('officials', 'o'),
 ('found', 'o'),
 ('eating', 'o'),
 ('cheezits', 'o'),
 ('under', 'o'),
 ('the', 'o'),
 ('bleachers', 'o')]

In [821]:
hmm_tagger.tag(twt.tokenize("Biden and Putin had lunch in Moscow, waiting for a call from Egypt"))

[('Biden', 'Country'),
 ('and', 'o'),
 ('Putin', 'Country'),
 ('had', 'o'),
 ('lunch', 'o'),
 ('in', 'o'),
 ('Moscow', 'Country'),
 (',', 'o'),
 ('waiting', 'o'),
 ('for', 'o'),
 ('a', 'o'),
 ('call', 'o'),
 ('from', 'o'),
 ('Egypt', 'Country')]

In [822]:
annotate("Biden and Putin had lunch in Moscow, waiting for a call from Egypt")

[('Biden', 'Country'),
 ('and', 'o'),
 ('Putin', 'Country'),
 ('had', 'o'),
 ('lunch', 'o'),
 ('in', 'o'),
 ('Moscow', 'Country'),
 (',', 'o'),
 ('waiting', 'o'),
 ('for', 'o'),
 ('a', 'o'),
 ('call', 'o'),
 ('from', 'o'),
 ('Egypt', 'Country')]

In [823]:
annotate('Turkish official says death toll from earthquake is now at 284, bringing overall death toll in Turkey and Syria to 568')

[('Turkish', 'Country'),
 ('official', 'o'),
 ('says', 'o'),
 ('death', 'o'),
 ('toll', 'o'),
 ('from', 'o'),
 ('earthquake', 'o'),
 ('is', 'o'),
 ('now', 'o'),
 ('at', 'o'),
 ('284', 'o'),
 (',', 'o'),
 ('bringing', 'o'),
 ('overall', 'o'),
 ('death', 'o'),
 ('toll', 'o'),
 ('in', 'o'),
 ('Turkey', 'Country'),
 ('and', 'o'),
 ('Syria', 'Country'),
 ('to', 'o'),
 ('568', 'o')]

In [824]:
hmm_tagger.tag(twt.tokenize('Turkish official says death toll from earthquake is now at 284, bringing overall death toll in Turkey and Syria to 568'))

[('Turkish', 'Country'),
 ('official', 'o'),
 ('says', 'o'),
 ('death', 'o'),
 ('toll', 'o'),
 ('from', 'o'),
 ('earthquake', 'o'),
 ('is', 'o'),
 ('now', 'o'),
 ('at', 'o'),
 ('284', 'o'),
 (',', 'o'),
 ('bringing', 'o'),
 ('overall', 'o'),
 ('death', 'o'),
 ('toll', 'o'),
 ('in', 'o'),
 ('Turkey', 'Country'),
 ('and', 'o'),
 ('Syria', 'Country'),
 ('to', 'o'),
 ('568', 'o')]

In [829]:
hmm_tagger.tag(twt.tokenize('Eighth Covid wave strikes Australia'))

[('Eighth', 'Country'),
 ('Covid', 'o'),
 ('wave', 'o'),
 ('strikes', 'o'),
 ('Australia', 'Country')]

In [830]:
twt.tokenize('Eighth Covid wave strikes Australia')

['Eighth', 'Covid', 'wave', 'strikes', 'Australia']